In [21]:
# import our libraries
import win32com.client
import requests

# get the active instance of Excel
ExcelApp = win32com.client.GetActiveObject("Excel.Application")

In [27]:
# define our API Key
API_KEY = ExcelApp.Workbooks(1).Worksheets("Sheet2").Range("A1").Value

# Define Base URL
final_url = 'https://www.googleapis.com/youtube/v3/playlistItems'

# Define search parameters
PARAMETERS = {'part': 'snippet,contentDetails,status',
              'maxResults': 5,
              'playlistId':'PLcFcktZ0wnNlRhiXWzu7Mkn2GC72f2LXp',
              'key': API_KEY}

# make a request to the Youtube API
response = requests.get(url = final_url, params = PARAMETERS)

# decode our JSON String
decoded_response = response.json()

In [35]:
def flatten_json(y):    
    out = {}
    
    def flatten(x, name = ''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
            
    flatten(y)
    return out

# flatten the dictionary
my_items = [flatten_json(video) for video in decoded_response['items']]

# create a list of column headers
column_headers = [key_value for key_value in my_items[0].keys()]

# define the worksheet
wrksht = ExcelApp.Workbooks(1).Worksheets("Sheet1")

# define the header section and populate it
start_cell = wrksht.Cells(1,1)
end_cell = wrksht.Cells(1, len(column_headers))
wrksht.Range(start_cell, end_cell).Value = column_headers

# get the row values
row_values = [[my_list for my_list in video.values()] for video in my_items]

# define the row section
start_cell = wrksht.Cells(2, 1)
end_cell = wrksht.Cells(1 + len(row_values), len(column_headers))

# populate the row values
wrksht.Range(start_cell, end_cell).Value = row_values
wrksht.Range(start_cell, end_cell).WrapText = False